# Installs and Imports

In [1]:
!pip install matplotlib
!pip install seaborn

In [2]:
# Fenics imports:
import dolfinx
import dolfinx.plot
import ufl

# Numerics imports:
from mpi4py import MPI
from petsc4py import PETSc
import numpy as np

# Visualisation imports:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

# Misc imports:
from math import sin, cos, pi, ceil, floor
import os
import json
import itertools
import time

In [3]:
# Set plotting settings:
mpl.rcParams['figure.dpi'] = 100
sns.set_style("darkgrid")

# FEM Code Functions

## Main Subroutine

In [41]:
def solve_linear_problem(mesh, y_rot, x_rot, E, lambda_, nu, rho, g, elem_order=2):

    V = dolfinx.VectorFunctionSpace(mesh, ("CG", elem_order))

    bcs = create_bcs(mesh, V)
    
    # Compute Lame parameters:
    mu_val = E/(2*(1 + nu))
    mu = dolfinx.Constant(mesh, mu_val)
    lambda_ = dolfinx.Constant(mesh, lambda_)

    u = ufl.TrialFunction(V)
    v = ufl.TestFunction(V)

    def epsilon(u):
        return ufl.sym(ufl.grad(u)) # Equivalent to 0.5*(ufl.nabla_grad(u) + ufl.nabla_grad(u).T)
    def sigma(u):
        return lambda_ * ufl.nabla_div(u) * ufl.Identity(u.geometric_dimension()) + 2*mu*epsilon(u)

    f = create_load_vector(g, rho, y_rot, x_rot, mesh)
    a = ufl.inner(sigma(u), epsilon(v)) * ufl.dx
    L = ufl.dot(f, v) * ufl.dx
    
    # Delete cache of previous models:
    try:
        !rm -r /root/.cache/fenics/*
    except:
        pass
    
    problem = dolfinx.fem.LinearProblem(a, L, bcs=bcs, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
    u = problem.solve()
    
    volumes = compute_volumes(u, mesh)
    
    return (u, volumes)

## Mesh + BCs

In [5]:
def create_mesh(L, W, NL, NW):
    mesh = dolfinx.BoxMesh(MPI.COMM_WORLD,[[0.0,0.0,0.0], [L, W, W]], [NL, NW, NW], dolfinx.cpp.mesh.CellType.hexahedron)
    return mesh

In [6]:
def create_bcs(mesh, V):
    fixed = lambda x: np.isclose(x[0], 0)
    fixed_facets = dolfinx.mesh.locate_entities_boundary(mesh, mesh.topology.dim - 1, fixed)
    facet_tag = dolfinx.MeshTags(mesh, mesh.topology.dim-1, fixed_facets, 1)
    u_bc = dolfinx.Function(V)
    with u_bc.vector.localForm() as loc:
        loc.set(0)
    left_dofs = dolfinx.fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.indices[facet_tag.values==1])
    bcs = [dolfinx.DirichletBC(u_bc, left_dofs)]
    return bcs  

In [7]:
def create_constitutive_eqn(E, nu, kappa, mesh, V, quad_degree=4):
    
    # Compute Lame parameters:
    mu = dolfinx.Constant(mesh, E/(2*(1 + nu)))
    
    # Compute new body force:
    B = dolfinx.Constant(mesh, (0,0,0))
    v = ufl.TestFunction(V)
    u = dolfinx.Function(V)
    d = len(u)
    I = ufl.variable(ufl.Identity(d))
    F = ufl.variable(I + ufl.grad(u))
    C = ufl.variable(F.T * F)
    J = ufl.variable(ufl.det(F))
    Ic = ufl.variable(ufl.tr(C))

    # Nearly-Incompressible Neo-Hookean material; 
    # See: https://link.springer.com/article/10.1007/s11071-015-2167-1
    psi = (mu/2)*(Ic-3) + kappa/2*(J-1)**2
    
    # Hyper-elasticity
    P = ufl.diff(psi, F)
    
    metadata = {"quadrature_degree": 4}
    dx = ufl.Measure("dx", metadata=metadata)

    # Define form F (we want to find u such that F(u) = 0)
    F = ufl.inner(ufl.grad(v), P)*dx - ufl.inner(v, B)*dx
    
    return (F, u, B)

## Loading Functions

In [8]:
def create_load_vector(g, rho, y_rot, x_rot, mesh, g_dir=(1,0,0)):
    rot_matrix = create_rot_matrix(y_rot, x_rot)
    f = rot_matrix @ (g*rho*np.array(g_dir))
    return dolfinx.Constant(mesh, f)
    
def create_rot_matrix(y_rot, x_rot, ang_to_rad=pi/180):
    # NB: Negative associated with y so increasing y_rot goes in 'right direction'
    theta, psi = -ang_to_rad*y_rot, ang_to_rad*x_rot
    rot_matrix = np.array([[         cos(theta),        0,          -sin(theta)],
                           [sin(psi)*sin(theta),  cos(psi), sin(psi)*cos(theta)],
                           [cos(psi)*sin(theta), -sin(psi), cos(psi)*cos(theta)]])
    return rot_matrix

## Pre-Processing Functions

In [9]:
def create_param_combos(**kwargs):
    keys = kwargs.keys()
    param_combos = []
    for bundle in itertools.product(*kwargs.values()):
        param_dict = dict(zip(keys, bundle))
        param_combos.append(param_dict)
    return param_combos

## Post-Processing Functions

In [10]:
def compute_volumes(u, mesh, quad_order=4):
    
    before_vol, after_vol = [], []
    ndim = mesh.geometry.x.shape[1]
    I = ufl.Identity(ndim)
    dx = ufl.Measure("dx", domain=mesh, metadata={"quadrature_degree": quad_order})
    const_funspace = dolfinx.VectorFunctionSpace(mesh, ("DG", 0), dim=1)
    const_fun = dolfinx.Function(const_funspace)
    const_fun.vector[:] = np.ones(const_fun.vector[:].shape)
    ufl.inner(const_fun,const_fun)
    vol_before = dolfinx.fem.assemble.assemble_scalar(ufl.inner(const_fun,const_fun)*dx)
    F = I + ufl.grad(u)
    vol_after = dolfinx.fem.assemble.assemble_scalar(ufl.det(F)*dx)
    
    return [vol_before, vol_after]

In [11]:
def get_end_displacement(u, mesh, W, L):
    u_vals = u.compute_point_values().real
    idx = np.isclose(mesh.geometry.x, [L, W, W])
    idx = np.where(np.all(idx, axis=1))
    u_vals = u_vals[idx]
    disp = np.sum(u_vals**2, axis=1)**(1/2)
    return disp.item()

# Selecting Lambda

In [35]:
def create_data(E_list, y_rot_list, x_rot_list, lambda_, nu, W, L, NL, NW, rho, g, verbose=False):
    
    mesh = create_mesh(L, W, NL, NW)  
    num_elem = NL * NW * NW
    
    # Get all possible combinations of parameters:
    param_combos = create_param_combos(E=E_list, y_rot=y_rot_list, x_rot=x_rot_list)
    results = {key: [] for key in ('E', 'y_rot', 'x_rot', 'disp', 'end_disp', 'time', 'volume')}
    
    for i, params in enumerate(param_combos):
        
        # Extract i'th set of parameters:
        E, y_rot, x_rot = params['E'], params['y_rot'], params['x_rot']
        
        if verbose:
            print(f"Simulating Mesh {i+1}/{len(param_combos)} (E = {E}, y_rot = {y_rot}, x_rot = {x_rot})")
        
        # Compute displacement of beam:
        t_start = time.time()
        u, volumes = solve_linear_problem(mesh, y_rot, x_rot, E, lambda_, nu, rho, g)
        t_end = time.time()
        t_solve = t_end - t_start
        
        if verbose:
            print(f'Simulation took {t_solve/60:.2f} mins.\n')
        
        # Get displacement field:
        disp = u.compute_point_values().real
        
        # Compute displacement at very end of beam:
        end_disp = get_end_displacement(u, mesh, W, L)
        
        # Save results:
        results['time'].append(t_solve)
        results['volume'].append(volumes)
        results['disp'].append(disp.tolist())
        results['end_disp'].append(end_disp)
        results['E'].append(E)
        results['y_rot'].append(y_rot)
        results['x_rot'].append(x_rot)
    
    # ALso save other simulation quantities:
    coords = mesh.geometry.x
    results['coords'] = coords.tolist()
    results['nu'] = nu
    results['density'] = rho
    results['W'] = W
    results['L'] = L
    results['NL'] = NL
    results['NW'] = NW
    results['num_elem'] = num_elem
    results['lambda'] = lambda_
    results['rho'] = rho
    
    return results

In [38]:
def perform_lambda_study(lambda_list, data, g=9.81, verbose=False):
    
    E_list, y_rot_list, x_rot_list = np.unique(data['E']), np.unique(data['y_rot']), np.unique(data['x_rot'])
    true_disp = np.array(data['end_disp'])
    W, L, NL, NW, rho, nu = data['W'], data['L'], data['NW'], data['NL'], data['rho'], data['nu']
    
    results = {key:[] for key in ('error', 'results', 'time')}
    for i, lambda_i in enumerate(lambda_list):
        if verbose:
            print(f"Simulating lambda value = {lambda_i} (Simulation {i+1}/{len(lambda_list)})")
        t_start = time.time()
        results_i = create_data(E_list, y_rot_list, x_rot_list, lambda_i, nu, W, L, NL, NW, rho, g, verbose=False)
        t_solve = t_start - time.time()
        results['results'].append(results_i)
        results['time'].append(t_solve)
        error = np.mean((results_i['end_disp'] - true_disp)**2).item()
        if verbose:
            print(f'Simulations took {t_solve/60:.2f} mins. Error = {error:.2f} \n')
        results['error'].append(error)
        
    return results

In [ ]:
with open('normal_nonlinear_train.json','r') as f:   
    nonlinear_data = json.load(f)

lambda_min, lambda_max = 1e1, 1e4  
num_pts = 2
lambda_list = np.linspace(lambda_min, lambda_max, num_pts).tolist()
lambda_results = perform_lambda_study(lambda_list, nonlinear_data, verbose=True)

with open('linear_lambda.json','w') as f:   
    json.dump(lambda_results, f, indent=4)

Simulating lambda value = 10.0 (Simulation 1/2)


In [ ]:
a

Based on the above plot, it appears that $\lambda$ =  best allows for the linear elasticity model to approxiamte the finite elasticity model.

In [ ]:
# Material constants:
nu = 0.33 # dimensionless
rho = 0.00102 # in g mm^-3
g = 9.81 # in m s^-2

# Beam dimensions:
W = 40 # in mm
L = 90 # in mm

# FEM parameters:
el_order = 2

# Store parameters in dictionary:
fixed_params = {"nu":nu, "rho":rho, "g":g, "el_order":el_order, "L":L, "W":W}

# Define stiffness and beam angle to perform convergence study for:
E = 10
beam_angle = 90

# Define number of elements to check:
num_elem_list = [100, 250, 500, 750, 1000, 1500, 2000, 2500, 3000, 3500, 4000]
    
# Create results dictionary:
convergence_results = mesh_convergence(num_elem_list, E, beam_angle, fixed_params)

# Save dictionary to JSON file:
save_name = "linear_convergence.json"
with open('./'+save_name, 'w') as f:
    json.dump(convergence_results, f, indent=4)

In [ ]:
# Mesh convergence plot:
import json
import matplotlib as mpl
from matplotlib import pyplot as plt
mpl.rcParams['figure.dpi'] = 200

with open('./linear_convergence.json', 'r') as f:
    convergence_results = json.load(f)
fig, ax = plt.subplots()
plt.plot(convergence_results['Number of Elements'], convergence_results['End Displacement'])
plt.title('Mesh Convergence Plot for Linear Elastic Beam', pad=15)
ax.set_xlabel('Number of Elements')
ax.set_ylabel('Beam Tip Displacement (mm)')
fig.patch.set_facecolor('white')
plt.plot(convergence_results['Number of Elements'], convergence_results['End Displacement'], 'x', color='black', markersize=5)
plt.show()

From above plot, looks like 3000 elements should be more than enough.

# Generate Data

##  Define Fixed Values

In [ ]:
nu = 0.33 # dimensionless
rho = 0.00102 # in g mm^-3
g = 9.81 # in m s^-2
W = 40 # in mm
L = 90 # in mm
elem_size = W/10
NL, NW = ceil(L/elem_size), ceil(W/elem_size)
lambda_ = 

## Generate and Plot Training Data

In [ ]:
# First create training data:
num_pts = 10
min_E, max_E = 10, 40
min_y_rot, max_y_rot = 0, 180
min_x_rot, max_x_rot = 0, 90
E_list = [E for E in np.linspace(min_E, max_E, num_pts)]
y_rot_list = [y for y in np.linspace(min_y_rot, max_y_rot, num_pts)]
# num_pts_x = int(num_pts/2)
# x_rot_list = [x for x in np.linspace(min_x_rot, max_x_rot, num_pts_x)]
x_rot_list = [0]

training_results = create_data(E_list, y_rot_list, x_rot_list, lambda_, nu, W, L, NL, NW, rho, g)

# Save dictionary to JSON file:
with open("normal_linear_train.json", 'w') as f:
    json.dump(training_results, f, indent=4)

## Generate and Plot Test Data

In [ ]:
# Next create test data:
delta_E = (max_E-min_E)/(num_pts-1)
delta_y_rot = (max_y_rot-min_y_rot)/(num_pts-1)
E_list = [E for E in np.linspace(min_E+0.5*delta_E, max_E-0.5*delta_E, num_pts-1)]
y_rot_list = [angle for angle in np.linspace(min_y_rot+0.5*delta_y_rot, max_y_rot-0.5*delta_y_rot, num_pts-1)]
# delta_x_rot = (max_x_rot-min_x_rot)/(num_pts_x-1)
# x_rot_list = [angle for angle in np.linspace(min_x_rot+0.5*delta_x_rot, max_x_rot-0.5*delta_x_rot, num_pts_x-1)]
x_rot_list = [0]

test_results = create_data(E_list, y_rot_list, x_rot_list, _lambda, nu, W, L, NL, NW, rho, g)

# Save dictionary to JSON file:
save_name = "linear_test.json"
with open('normal_linear_test.json', 'w') as f:
    json.dump(test_results, f, indent=4)

# Plot Data

In [ ]:
# Let's plot the training and test data:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

mpl.rcParams['figure.dpi'] = 200

def plot_data(data, grid_shape, num_levels, E_lims=None, angle_lims=None, y_lims=None, title=None):

    angle, E, y = np.array(data['Beam Angle']), np.array(data['E']), np.array(data['End Displacement'])
    
    levels = np.linspace(y_lims[0], y_lims[1], num_levels)
    
    # Create surface plot:
    fig, ax = plt.subplots()
    contour_fig = ax.contourf(E.reshape(grid_shape), angle.reshape(grid_shape), y.reshape(grid_shape), 
                              levels=levels, cmap=cm.coolwarm)
    
    ticks = np.array(range(y_lims[0], y_lims[1]+1, 20))
    cbar = fig.colorbar(contour_fig, ticks=ticks)
    cbar.set_label('Beam Tip Displacement (mm)', rotation=270, labelpad=15)
    ax.set_xlabel("Young's Modulus (kPa)")
    ax.set_ylabel('Beam Angle (degrees)')
    ax.set_xlim(E_lims)
    ax.set_ylim(angle_lims)
    plt.title(title, pad=15)
    plt.plot(E, angle, 'x', color='black', markersize=3)
    fig.patch.set_facecolor('white')
    plt.show()

In [ ]:
with open('linear_train.json', 'r') as f:
    training_dict = json.load(f)

y_lims = [0, 80]
E_lims = [10, 40]
angle_lims = [0, 180]
grid_shape = (10, 10)
num_levels = 100
title = "Linear Elastic Beam Training Data"
plot_data(training_dict, grid_shape, num_levels, E_lims, angle_lims, y_lims, title)

In [ ]:
with open('linear_test.json', 'r') as f:
    test_dict = json.load(f)

y_lims = [0, 80]
E_lims = [10, 40]
angle_lims = [0, 180]
grid_shape = (9, 9)
num_levels = 100
title = "Linear Elastic Beam Test Data"
plot_data(test_dict, grid_shape, num_levels, E_lims, angle_lims, y_lims, title)